[View in Colaboratory](https://colab.research.google.com/github/ObaidAshraf/Keras_models/blob/master/DL_P2_MultiClassClassification.ipynb)

## **Step 1. Project Description**

In [2]:
from google.colab import files
uploaded = files.upload() 

Saving iris.csv to iris.csv


## **Step 2. Making Preparations**

In [0]:
import numpy
import pandas
import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# load dataset
dataframe = pandas.read_csv("iris.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]

le = LabelEncoder()
encoded_Y = le.fit(Y)
encoded_Y = le.transform(Y)
encoded_Y = np_utils.to_categorical(encoded_Y)
train_data = X
train_targets = encoded_Y

## **Step 3: Define the Neural Network Baseline Model**

In [0]:
def baseline_model():
  model = Sequential()
  model.add(Dense(8, activation = 'relu', input_shape=(4,)))
  model.add(Dense(3, activation = 'softmax'))
  
  model.compile(loss = 'categorical_crossentropy',
                optimizer = 'adam',
                metrics = ['accuracy'])
  
  return model

## **Step 4. Evaluate The Model with k-Fold Cross Validation**

In [8]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))



Results: 97.33% (4.42%)


## Step 5.1. Evaluate a Smaller Network

In [0]:
def smaller_model():
  model = Sequential()
  model.add(Dense(8, activation = 'relu', input_shape=(4,)))
  model.add(Dense(4, activation = 'relu'))
  model.add(Dense(3, activation = 'softmax'))
  
  model.compile(loss = 'categorical_crossentropy',
                optimizer = 'adam',
                metrics = ['accuracy'])
  
  return model

In [10]:
estimator = KerasClassifier(build_fn=smaller_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))



Results: 96.67% (4.47%)


## Step 5.2. Evaluate a Larger Network

In [0]:
def larger_model():
  model = Sequential()
  model.add(Dense(8, activation = 'relu', input_shape=(4,)))
  model.add(Dense(8, activation = 'relu'))
  model.add(Dense(4, activation = 'relu'))
  model.add(Dense(3, activation = 'softmax'))
  
  model.compile(loss = 'categorical_crossentropy',
                optimizer = 'adam',
                metrics = ['accuracy'])
  
  return model

In [12]:
estimator = KerasClassifier(build_fn=larger_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))



Results: 92.00% (13.60%)


## Step 6. Really Scaling up: developing a model that overfits & Step 7. Tuning the Model

In [0]:
def tuned_model():
  model = Sequential()
  model.add(Dense(8, activation = 'relu', input_shape=(4,)))
  model.add(Dense(8, activation = 'relu'))
  model.add(Dense(8, activation = 'relu'))
  model.add(Dense(5, activation = 'relu'))
  model.add(Dense(3, activation = 'softmax'))
  
  model.compile(loss = 'categorical_crossentropy',
                optimizer = 'adam',
                metrics = ['accuracy'])
  
  return model

In [14]:
estimator = KerasClassifier(build_fn=tuned_model, epochs=250, batch_size=5, verbose=0)
kfold = KFold(n_splits=25, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))



Results: 92.67% (20.59%)


## Step 8. Rewriting the code using the Keras Functional API

In [0]:
def create_func_api_model():
  inputs = keras.Input(shape=(4,))
  x = layers.Dense(8, activation = 'relu')(inputs)
  x = layers.Dense(8, activation = 'relu')(x)
  x = layers.Dense(8, activation = 'relu')(x)
  x = layers.Dense(5, activation = 'relu')(x)
  outputs = layers.Dense(3, activation = 'softmax')(x)
  
  model = keras.Model(inputs, outputs)
  model.compile(loss = 'categorical_crossentropy',
                optimizer = 'adam',
                metrics = ['accuracy'])
  return model

In [5]:
estimator = KerasClassifier(build_fn=create_func_api_model, epochs=250, batch_size=5, verbose=0)
kfold = KFold(n_splits=25, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))



Results: 96.00% (7.12%)


## Step 9. Rewriting the code by doing Model Subclassing

In [0]:
# TODO

## Step 10. Rewriting the code without using scikit-learn

In [0]:
def baseline_model():
  model = Sequential()
  model.add(Dense(8, activation = 'relu', input_shape=(4,)))
  model.add(Dense(8, activation = 'relu'))
  model.add(Dense(8, activation = 'relu'))
  model.add(Dense(5, activation = 'relu'))
  model.add(Dense(3, activation = 'softmax'))
  
  model.compile(loss = 'categorical_crossentropy',
                optimizer = 'adam',
                metrics = ['accuracy'])
  
  return model

In [9]:
import numpy as np

k=5
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []

num_epochs = 500
batch_size_val = num_val_samples // k
all_mae_histories = []
for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)
    
    model = baseline_model()
    history = model.fit(partial_train_data, partial_train_targets,
                    validation_data=(val_data, val_targets),
                    epochs=num_epochs, batch_size=batch_size_val, verbose=0)
    print("Training Accuracy:", history.history['acc'][num_epochs-1])
    print("Validation Accuracy:", history.history['val_acc'][num_epochs-1])


processing fold # 0
Training Accuracy: 0.9749999970197678
Validation Accuracy: 1.0
processing fold # 1
Training Accuracy: 0.9833333343267441
Validation Accuracy: 1.0
processing fold # 2
Training Accuracy: 1.0
Validation Accuracy: 0.8333333373069763
processing fold # 3
Training Accuracy: 0.9833333313465118
Validation Accuracy: 0.9666666626930237
processing fold # 4
Training Accuracy: 1.0
Validation Accuracy: 0.8
